In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from scipy.io import savemat
import time

# Define the dimensions of the rectangular surface
Lx = 50 # length in x-direction
Ly = 50 # length in y-direction

# Define the dimensions of the ellipses
a = 4*0.25 # major axis
b = 0.25 # minor axis
theta = np.linspace(0, 2*np.pi, 20)

# Initialize a list to store the ellipses
ellipses = []

# Load existing ellipses if any
# load('AR1_repeat.mat')

# Define the maximum distance for checking overlap 
max_dist = 2*a

# Define the filename to save the centroids
filename = 'centroids.mat'

# Define the interval to save the centroids (in seconds)
interval = 10*60

# Initialize the time of the last save
last_save_time = time.time()

# Loop over the number of ellipses to place
for i in range(len(ellipses), 1000000):
    
    # Initialize a flag to indicate whether the ellipse has been successfully placed
    placed = False
    
    # Loop until the ellipse is successfully placed
    while not placed:
        
        # Generate a random position for the center of the ellipse
        x = np.random.uniform(a, Lx - a)
        y = np.random.uniform(b, Ly - b)
        
        # Create a Polygon object for the ellipse
        xe = a*np.cos(theta) + x
        ye = b*np.sin(theta) + y
        ellipse = Polygon(np.column_stack((xe, ye)))
        
        # Check if the ellipse overlaps with any existing ellipses within a certain distance
        overlap = False
        for j in range(len(ellipses)):
            if Point(ellipse.centroid).distance(Point(ellipses[j].centroid)) <= max_dist and ellipse.intersects(ellipses[j]):
                overlap = True
                break
        
        # If the ellipse does not overlap, add it to the list of ellipses and increment the counter
        if not overlap:
            ellipses.append(ellipse)
            placed = True
            print(f'Ellipses placed: {len(ellipses)}')
            # Save the centroids every 10 seconds
            if time.time() - last_save_time >= interval:
                centroids = []
                for j in range(sum([1 for e in ellipses if e])):
                    e = ellipses[j]
                    p = Polygon(e)
                    x, y = p.centroid.x, p.centroid.y
                    centroids.append([1, x, y])
                centroids = np.array(centroids)
                savemat(filename, {'centroids': centroids})
                last_save_time = time.time()

# Save the final centroids
centroids = []
for i in range(sum([1 for e in ellipses if e])):
    e = ellipses[i]
    p = Polygon(e)
    x, y = p.centroid.x, p.centroid.y
    centroids.append([1, x, y])
centroids = np.array(centroids)
savemat(filename, {'centroids': centroids})

# Plot the ellipses
fig, ax = plt.subplots(figsize=(10, 10))
for ellipse in ellipses:
    ax.plot(*ellipse.exterior.xy, color='black')
ax.set_xlim(0, Lx)


In [ ]:
melipses=ellipses

In [ ]:
del ellipses

In [ ]:
ellipses=melipses

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
for ellipse in ellipses:
    ax.plot(*ellipse.exterior.xy, color='black')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
plt.show()

In [ ]:
import numpy as np
from shapely.geometry import Polygon, Point

# Load matrix V from a file

# Define the dimensions of the rectangular surface
Lx = 50
Ly = 50

# Define the dimensions of the ellipses
a = 4*0.25 # major axis
b = 0.25 # minor axis
theta = np.linspace(0, 2*np.pi, 20)

# Initialize a list to store the ellipses to be added to V
new_ellipses = []

# Loop over the mesh positions
for i in np.arange(0, Lx, 0.05):
    for j in np.arange(0, Ly, 0.05):
        # Compute the center of the mesh
        x = i + 0.025
        y = j + 0.025
        
        # Create a Polygon object for the ellipse
        xe = a*np.cos(theta) + x
        ye = b*np.sin(theta) + y
        ellipse = Polygon(np.column_stack((xe, ye)))
        
        # Check if the ellipse overlaps with any existing ellipses within a certain distance
        overlap = False
        for j in range(len(ellipses)):
            if Point(ellipse.centroid).distance(Point(ellipses[j].centroid)) <= max_dist and ellipse.intersects(ellipses[j]):
                overlap = True
                break
        
        # If the ellipse does not overlap and is within the boundary of the box, add it to the list of ellipses and increment the counter
        if not overlap and ellipse.within(Polygon([(0,0), (0,Ly), (Lx,Ly), (Lx,0)])):
            ellipses.append(ellipse)
            placed = True
            print(f'Ellipses placed: {len(ellipses)}')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
for ellipse in ellipses:
    ax.plot(*ellipse.exterior.xy, color='black')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
plt.show()

In [ ]:
import numpy as np
from shapely.geometry import Polygon, Point
from scipy.spatial import cKDTree

# Define the dimensions of the rectangular surface
Lx = 50
Ly = 50

# Define the dimensions of the ellipses
a = 4*0.25 # major axis
b = 0.25 # minor axis
theta = np.linspace(0, 2*np.pi, 20)

# Initialize an empty list to store the ellipses
newellipses = []

# Calculate the centers of all the meshes
xv, yv = np.meshgrid(np.arange(0, Lx, 0.05) + 0.025, np.arange(0, Ly, 0.05) + 0.025)

# Create an array to store the ellipse centers
centers = np.column_stack((xv.ravel(), yv.ravel()))

# Create an array to store the ellipse coordinates
ell_coords = np.column_stack((a*np.cos(theta), b*np.sin(theta)))

# Loop over each center and create an ellipse
for center in centers:
    # Create a Polygon object for the ellipse
    xe, ye = ell_coords[:,0] + center[0], ell_coords[:,1] + center[1]
    ellipse = Polygon(np.column_stack((xe, ye)))
    
    # Check if the ellipse overlaps with any existing ellipses within a certain distance
    overlap = False
    if len(ellipses) > 0:
        tree = cKDTree([e.centroid.coords[0] for e in ellipses])
        distances, indices = tree.query([ellipse.centroid.coords[0]], k=10, distance_upper_bound=0.5)
        for idx in indices[0]:
            if idx < len(ellipses) and ellipse.intersects(ellipses[idx]):
                overlap = True
                break
    
    # If the ellipse does not overlap and is within the boundary of the box, add it to the list of ellipses
    if not overlap and ellipse.within(Polygon([(0,0), (0,Ly), (Lx,Ly), (Lx,0)])):
        ellipses.append(ellipse)
        print(f'Ellipses placed: {len(ellipses)}')

# Print the number of ellipses placed
print(f'Ellipses placed: {len(ellipses)}')


In [ ]:
import numpy as np
from scipy.io import savemat

# Calculate ellipse centroids
centroids = []
for i in range(sum([1 for e in ellipses if e])):
    ellipse = ellipses[i]
    ellipse = Polygon(ellipse)
    x, y = ellipse.centroid.x, ellipse.centroid.y
    centroids.append([1, x, y])
centroids = np.array(centroids)

# Save centroids as MATLAB file
savemat('matrix_disc_20by20.mat', {'centroids': centroids})

In [ ]:
from google.colab import files

files.download('matrix_disc_20by20.mat')